# Boulder, Colorado Urban Greenspace

Boulder, Colorado is a city in the foothills of the Front Range of the Rocky Mountains in the U.S. state of Colorado. It has a semi-arid climate with cold winters and warm, relatively wet summers. The City has been considered a leader in urban forestry being named a Tree City USA by the National Arbor Day Foundation since 1984 [1]. The city maintains a street tree inventory. In 2018, the City of Boulder released the Urban Forest Strategic Plan and in 2023, the City issued a State of the Urban Forest Report [2]. The Strategic Plan outlines a goal to maintain the city's tree canopy at 16% (a no-net-loss goal) given ongoing declines in the tree canopy due to the emerald ash borer.

In this notebook, I examined greenspace across the City of Boulder by census tract and conducted a linear regression analysis to test if median annual income correlates with the fraction of greenspace.

<img src="https://upload.wikimedia.org/wikipedia/commons/7/7e/BoulderBearPeak.jpg" alt="Image of Boulder, Colorado" width="400">

> Sources:
>
> [1] https://bouldercolorado.gov/government/departments/forestry/about#main-content
>
> [2] https://storymaps.arcgis.com/stories/0cb784ee805144428f914f904a0bb367

In [1]:
# Standard imports
import getpass
import io
import os

# Non-standard imports
import cartopy.crs as ccrs
from census import Census
import earthpy as et
import geoviews as gv
import geopandas as gpd

import hvplot
import holoviews as hv
import hvplot.pandas
import hvplot.xarray

import matplotlib.pyplot as plt
import numpy as np

import pystac_client
import pandas as pd

import requests
import rioxarray as rxr
from rioxarray.merge import merge_arrays

import shapely

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

import us

import warnings

import xarray as xr

warnings.filterwarnings('ignore')

In [2]:
data_dir = os.path.join(et.io.HOME, et.io.DATA_NAME, 'boulder-greenspace')

if not os.path.exists(data_dir):
    os.makedirs(data_dir)


In [37]:
# %%bash
# ls ~/earth-analytics/data/boulder-greenspace

## Download data

**Data citations**

> “City of Boulder Open Data.” n.d. City of Boulder City Limits. Accessed April 18, 2024. https://open-data.bouldercolorado.gov/.

> Bureau, US Census. n.d. “TIGER/Line Shapefiles.” Census.Gov. Accessed April 18, 2024. https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.html.

> Bureau, US Census. n.d. “American Community Survey (ACS).” Census.Gov. Accessed April 18, 2024. https://www.census.gov/programs-surveys/acs.

> “National Agriculture Imagery Program - NAIP Hub Site.” n.d. Accessed April 18, 2024. https://naip-usdaonline.hub.arcgis.com/.


In [4]:
# Download City of Boulder boundary

boundary_url = ("https://gis.bouldercolorado.gov/ags_svr1/rest/services/plan/CityLimits/MapServer/0/query?outFields=*&where=1%3D1&f=geojson")

boundary_path = os.path.join(data_dir, 'city_boundary.geojson')

if os.path.exists(boundary_path):

    boundary_gdf = gpd.read_file(boundary_path)
    print("Data is already downloaded.")

else:
    # Mimic web browser
    user_agent = (
        'Mozilla/5.0 (X11; Linux x86_64; rv:60.0) '
        'Gecko/20100101 Firefox/81.0'
    )

    # Download GEOJSON
    r = requests.get(url=boundary_url, headers={'User-Agent': user_agent})

    # Read GeoJSON data into a GeoDataFrame
    boundary_geojson_data = r.json()

    # Cache
    boundary_gdf = (gpd.GeoDataFrame
                    .from_features(boundary_geojson_data['features'])
                    .to_file(boundary_path, driver="GeoJSON")
                    )
    
    # Read
    boundary_gdf = gpd.read_file(boundary_path)
    print("Data downloaded and loaded.")



Data is already downloaded.


In [5]:
boundary_gdf.head()

,OBJECTID,TYPE,Shape.STArea(),Shape.STLength(),geometry
0,38,City,7.138182e+07,45971.753983,"POLYGON ((-105.20685 40.07559, -105.20685 40.0..."
1,39,City,4.073072e+07,64816.989082,"POLYGON ((-105.17870 40.06914, -105.17836 40.0..."
2,40,City,8.103168e+04,1661.732555,"POLYGON ((-105.21128 40.01726, -105.21138 40.0..."
3,41,City,2.092505e+07,34856.338250,"POLYGON ((-105.26349 40.08020, -105.26687 40.0..."
4,56,City,6.460669e+08,308304.306022,"POLYGON ((-105.22346 39.98856, -105.22360 39.9..."


In [6]:
# Test plot
# boundary_gdf.to_crs("4326").plot()

### Map of study area

In [7]:
boundary_gdf.hvplot(
    geo=True,
    tiles='EsriImagery',
    xlim=(-130, -60),  # Longitude bounds for the US
    ylim=(24, 50) 
)

lat = 40.022367
lon = -105.269461

In [8]:
point = gpd.GeoDataFrame(geometry=gpd.points_from_xy([-105], [40]))

# Plot the single point using hvplot
plot = (point.hvplot(geo=True, tiles='CartoLight', color='red',
                    width=500, height=400, xlim=(-130, -60), ylim=(24, 50))
.opts(title="Location of the City of Boulder within the United States")
)

# Show the plot
plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]

In [9]:
# Download census tracts
# print(us.states.CO.fips)

colorado_tracts_url = ("https://www2.census.gov/geo/tiger/TIGER2023/"
                       "TRACT/tl_2023_08_tract.zip")

colorado_tracts_path = os.path.join(data_dir, 'colorado_census_tracts.geojson')

if os.path.exists(colorado_tracts_path):
    colorado_tracts_gdf = gpd.read_file(colorado_tracts_path).to_crs("4326")
    print("Data is already downloaded.")
else:
    gpd.read_file(colorado_tracts_url).to_file(colorado_tracts_path, driver="GeoJSON")
    colorado_tracts_gdf = gpd.read_file(colorado_tracts_path).to_crs("4326")

08
Data is already downloaded.


In [69]:
# colorado_tracts_gdf.plot(edgecolor="white", linewidth=0.5)

In [68]:
# Select only the census tracts in Boulder
# print(colorado_tracts_gdf.crs)
# print(boundary_gdf.crs)

boulder_tracts_gdf = gpd.sjoin(colorado_tracts_gdf, boundary_gdf, how="inner", predicate="intersects")
# boulder_tracts_gdf.plot(edgecolor="white", linewidth=0.5)

# Clip to city boundary 
boulder_city_tracts_gdf = gpd.clip(boulder_tracts_gdf, boundary_gdf)
# boulder_city_tracts_gdf.plot(edgecolor="white", linewidth=0.5)

In [12]:
# Download census data

# B06011_001E: Estimate!!Median income in the past 12 months --!!Total:	
# Sources: https://api.census.gov/data/2019/acs/acs5/variables.html; https://pypi.org/project/census/

# Obtain Census variables from the 2019 American Community Survey at the tract level
census_path = os.path.join(data_dir, 'census_data_for_colorado.csv')

if os.path.exists(census_path):
    census_df = pd.read_csv(census_path)
    print("Data is already downloaded.")

else:
    # Authenticate
    api_key = getpass.getpass('U.S. Census API Key')
    c = Census(api_key)
    c

    CO_census = c.acs5.state_county_tract(fields = ('NAME', 'B06011_001E'),
                                          state_fips = '08',
                                          county_fips = "*",
                                          tract = "*",
                                          year = 2021)
    
    tracts_census_df = (pd.DataFrame(CO_census, 
                                     columns=['NAME', 
                                              'B06011_001E', 
                                              'state', 
                                              'county', 
                                              'tract'])
                        .to_csv(census_path, index=False))
    
    census_df = pd.read_csv(census_path)


census_df = census_df.rename(columns={'B06011_001E': 'median_income'})
census_df.head()

# Merge census data with tracts

boulder_city_tracts_gdf.head()

Data is already downloaded.


,STATEFP,COUNTYFP,TRACTCE,GEOID,GEOIDFQ,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,index_right,OBJECTID,TYPE,Shape.STArea(),Shape.STLength()
384,08,013,012510,08013012510,1400000US08013012510,125.10,Census Tract 125.10,G5020,S,50711625,87793,+39.9529586,-105.2704198,"POLYGON ((-105.28596 39.97229, -105.28587 39.9...",4,56,City,6.460669e+08,308304.306022
242,08,013,012509,08013012509,1400000US08013012509,125.09,Census Tract 125.09,G5020,S,1764488,51241,+39.9779451,-105.2504381,"POLYGON ((-105.26007 39.98007, -105.25993 39.9...",4,56,City,6.460669e+08,308304.306022
231,08,013,012505,08013012505,1400000US08013012505,125.05,Census Tract 125.05,G5020,S,15085912,10340,+39.9870088,-105.2999647,"MULTIPOLYGON (((-105.29124 40.00179, -105.2911...",4,56,City,6.460669e+08,308304.306022
114,08,013,012609,08013012609,1400000US08013012609,126.09,Census Tract 126.09,G5020,S,332661,0,+40.0036607,-105.2507561,"POLYGON ((-105.25310 40.00742, -105.25258 40.0...",4,56,City,6.460669e+08,308304.306022
113,08,013,012610,08013012610,1400000US08013012610,126.10,Census Tract 126.10,G5020,S,390612,0,+40.0040178,-105.2558459,"POLYGON ((-105.25835 40.00813, -105.25695 40.0...",4,56,City,6.460669e+08,308304.306022


In [40]:
# Merge datasets

boulder_city_tracts_gdf = boulder_city_tracts_gdf.loc[:, [
    'TRACTCE', 'geometry', 'NAMELSAD']]

boulder_city_tracts_gdf['TRACTCE'] = pd.to_numeric(
    boulder_city_tracts_gdf['TRACTCE'], errors='coerce')

tracts_w_census_gdf = (boulder_city_tracts_gdf.merge(
    census_df, left_on='TRACTCE', right_on='tract'))

negative_or_not = (tracts_w_census_gdf['median_income'] >= 0)

tracts_w_census_gdf = tracts_w_census_gdf[negative_or_not]
tracts_w_census_gdf.head()

,TRACTCE,geometry,NAMELSAD,NAME,median_income,state,county,tract
0,12510,"POLYGON ((-105.28596 39.97229, -105.28587 39.9...",Census Tract 125.10,"Census Tract 125.10, Boulder County, Colorado",74638.0,8,13,12510
1,12509,"POLYGON ((-105.26007 39.98007, -105.25993 39.9...",Census Tract 125.09,"Census Tract 125.09, Boulder County, Colorado",68521.0,8,13,12509
2,12505,"MULTIPOLYGON (((-105.29124 40.00179, -105.2911...",Census Tract 125.05,"Census Tract 125.05, Boulder County, Colorado",83926.0,8,13,12505
3,12609,"POLYGON ((-105.25310 40.00742, -105.25258 40.0...",Census Tract 126.09,"Census Tract 126.09, Boulder County, Colorado",17320.0,8,13,12609
4,12610,"POLYGON ((-105.25835 40.00813, -105.25695 40.0...",Census Tract 126.10,"Census Tract 126.10, Boulder County, Colorado",10366.0,8,13,12610


In [42]:
# tracts_w_census_gdf['median_income'].plot.hist()

In [44]:
# tracts_w_census_gdf.plot('median_income', legend=True)

In [45]:
# Test whether tracts are correctly georeferenced

tracts_basemap_plot = (tracts_w_census_gdf.hvplot(
    geo=True, alpha=0.3, tiles='EsriImagery')
    .opts(title="Census tracts in the City of Boulder",
          height=500,
          width=800)
)

tracts_basemap_plot

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]

In [67]:
# Download data using Microsoft Planetary Computer STAC catalog

# Access catalog
pc_catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1"
)

# pc_catalog.title

In [47]:
# Check if any data has already been downloaded and processed

all_greenspace_stats_path = os.path.join(data_dir, 'all_greenspace_stats.csv')

if os.path.exists(all_greenspace_stats_path):
    print("All greenspace stats file exists. Checking "
          "which tracts have been downloaded and processed...")

    # Extract tracts to be calculated
    all_tracts = tracts_w_census_gdf['TRACTCE'].astype('int64')

    # Load tracts already processed
    all_greenspace_stats_df = pd.read_csv(all_greenspace_stats_path)

    # Check which values in tract_numbers are not downloaded
    missing_tracts = all_tracts[~all_tracts.isin(all_greenspace_stats_df['tract'])]

    # Create a DataFrame with the missing tracts and their geometries
    tract_geometry = tracts_w_census_gdf[['TRACTCE', 'geometry']]
    tract_geometry['TRACTCE'] = tract_geometry['TRACTCE'].astype('int64')

    missing_tracts_df = pd.DataFrame({'missing_tract': missing_tracts.astype('int64')})

    missing_tracts_merged_df = missing_tracts_df.merge(
        tract_geometry,
        left_on='missing_tract',
        right_on='TRACTCE',
        how='left'
    )

    missing_tracts_gdf = gpd.GeoDataFrame(missing_tracts_merged_df, geometry='geometry')

    missing_tracts_gdf.head()

    # # Plot unprocessed tracts
    # ax = missing_tracts_gdf.plot(color='purple', label='Unprocessed tracts')

    # # Adding title
    # plt.title('Unprocessed tracts')

    # plt.show()

else:
    all_greenspace_stats_df = pd.DataFrame(columns=['tract', 'fraction_greenspace'])
    all_greenspace_stats_df.to_csv(all_greenspace_stats_path, index=False)

    print("All greenspace stats file does not exist. Created file.")
    


All greenspace stats file exists. Checking which tracts have been downloaded and processed...


In [49]:
# Compile image URLs for all tracts that are yet to be processed

naip_image_urls_path = os.path.join(data_dir, 'naip_image_urls.csv')
year = 2021
item_url_dfs = []

# Search for image for each tract
for index, tract in missing_tracts_gdf.iterrows():
    # print(tract) 

    tract_name = tract['TRACTCE']

    # print(tract_name)

    # Search catalog for image
    tract_geometry = tract['geometry']

    naip_search = pc_catalog.search(
    collections=["naip"],
    intersects=shapely.to_geojson(tract_geometry),
    datetime=f"{year}"
    )

    # print(naip_search)

    try:

        for naip_item in naip_search.items():

            # print(naip_item.id)
            item_url_dfs.append(
                pd.DataFrame(dict(
                    tract=[tract_name],
                    tile_id=[naip_item.id],
                    url=naip_item.assets['image'].href
                ))
            )
        
    except Exception as e:
        # print(f"Error processing item: {str(e)}")
            
        continue


item_url_df = pd.concat(item_url_dfs)

# Save URLS
item_url_df.to_csv(naip_image_urls_path, index=False)

In [50]:
# Download and process data for all tracts

ndvi_threshold = 0.12

all_greenspace_stats = []

# # Calculate greenspace fraction for each tract
# for tract, tract_urls in item_url_df.groupby('tract'):

#     print(f'tract: {tract}')

#     # Store all NDVI images for tract
#     tract_ndvi_das = []

#     try: 

#         for index, image in tract_urls.iterrows():
#             print("URL is:", image.url)

#             # Open NAIP data array
#             full_naip_vda = rxr.open_rasterio(image.url, masked=True).squeeze()

#             # Get census tract boundary
#             boundary_gdf = missing_tracts_gdf.to_crs(full_naip_vda.rio.crs)[missing_tracts_gdf.TRACTCE==tract]
            
#             # Clip NAIP data to boundary
#             crop_naip_vda = full_naip_vda.rio.clip_box(
#                 *boundary_gdf.total_bounds
#             )

#             naip_vda = crop_naip_vda.rio.clip(boundary_gdf.geometry)

#             # Compute NDVI
#             # Band 4: NIR, Band 1: Red
#             tract_ndvi_das.append(
#                 (naip_vda.sel(band=4) - naip_vda.sel(band=1))
#                 / (naip_vda.sel(band=4) + naip_vda.sel(band=1))
#             )

#         # Merge rasters if there are multiple images
#         if len(tract_ndvi_das)>1:
#             tract_ndvi_da = merge_arrays(tract_ndvi_das)
#             print("Merged images")

#         else:
#             print("Only one image for tract")
#             tract_ndvi_da = tract_ndvi_das[0]

#         # Compute fraction of greenspace (NDVI>NDVI threshold)
#         fraction_greenspace_da = np.sum(tract_ndvi_da > ndvi_threshold) / tract_ndvi_da.notnull().sum()

#         # Extract fraction
#         if fraction_greenspace_da.size == 1:
#             fraction_greenspace = fraction_greenspace_da.values.flatten()[0]
#             print(fraction_greenspace)

#         else:
#             print("Error: The fraction greenspace array has multiple values.")

#         # Add to accumulator list
#         tract_stats = [tract, fraction_greenspace]
#         all_greenspace_stats.append(tract_stats)
    
#     except Exception as e:
#             print(f"Error processing tract: {e}")
#             continue  # Continue to next tract

In [52]:
# Join tracts with geometry

newly_processed_tract_stats = pd.DataFrame(all_greenspace_stats, columns=all_greenspace_stats_df.columns)

# Add the newly processed stats to the original df
all_greenspace_stats_new_df = pd.concat([all_greenspace_stats_df, newly_processed_tract_stats], ignore_index=True)

# Cache
all_greenspace_stats_new_df.to_csv(all_greenspace_stats_path, index=False)

# Retrieve geometries
greenspace_gdf = pd.merge(tracts_w_census_gdf[['tract', 'geometry', 'median_income', 'NAME']], all_greenspace_stats_new_df, left_on='tract', right_on='tract', how='left')
# greenspace_gdf.head()

In [53]:
greenspace_plot = (greenspace_gdf.hvplot(geo=True, hover_cols=['value'], cmap='viridis_r', c='fraction_greenspace', width=500, height=500)
                   .opts(title='Greenspace fraction by census tract in Boulder, CO')
)

median_income_plot = (tracts_w_census_gdf.hvplot(geo=True, hover_cols=['value'], cmap='viridis_r', c='median_income', width=500, height=500)
                   .opts(title='Median income by census tract in Boulder, CO'))

greenspace_income_plot = greenspace_plot + median_income_plot

greenspace_income_plot

:Layout
   .Polygons.I  :Polygons   [Longitude,Latitude]   (fraction_greenspace)
   .Polygons.II :Polygons   [Longitude,Latitude]   (median_income)

In this map, where darker colors represent more greenspace fraction (left) and higher median household income (right) we can see that there is not a clear visual relationship between median income and the fraction of greenspace in the city. There is more greenspace in the western and southern portions of the city, which is consistent with the fact that these areas are adjacent to wildlands with naturally occurring forest.

## Linear ordinary least-squares regression


Statistics can provide us with a quantitative insight into whether median household income is correlated with higher greenspace fraction in Boulder.

In [57]:
# Create df for analysis
greenspace_values = greenspace_gdf.dropna()
greenspace_values = greenspace_values[['tract', 'fraction_greenspace']]
greenspace_values['tract'] = greenspace_values['tract'].astype('int64')

income_values = tracts_w_census_gdf[['tract', 'median_income']]

analysis_df = (pd.merge(greenspace_values, income_values, on='tract', how='left')
)

analysis_df.head()

,tract,fraction_greenspace,median_income
0,12510,0.619154,74638.0
1,12509,0.524220,68521.0
2,12505,0.748752,83926.0
3,12609,0.534308,17320.0
4,12610,0.258146,10366.0


In [58]:
scatter_plot = (analysis_df.hvplot.scatter(x='median_income', y='fraction_greenspace')
                       .opts(title='Median income versus greenspace fraction by census tract')
)
scatter_plot

:Scatter   [median_income]   (fraction_greenspace)

In [25]:
# See histogram to determine is log-transformation is needed
# analysis_df.hvplot.hist(y='fraction_greenspace') + analysis_df.hvplot.hist(y='median_income')

Fraction greenspace looks roughly normal. Median income seems skewed so we can log transform.

In [59]:
analysis_df['log_median_income'] = np.log(analysis_df['median_income'])

analysis_df.dropna(inplace=True)

In [27]:
# See histogram
# analysis_df.hvplot.hist(y='log_median_income')

## Linear regression analysis

In [60]:
X = analysis_df[['log_median_income']]
y = analysis_df[['fraction_greenspace']]

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.5, 
                                                    random_state=42)

In [61]:
# Fit linear regression to training data

linear_reg = sklearn.linear_model.LinearRegression().fit(X_train, y_train)
linear_reg.coef_

test_df = X_test.copy()
test_df['measured'] = y_test
test_df['predicted'] = linear_reg.predict(X_test)
y_max = float(y_test.max())

test_df_antilog = test_df.reset_index()

test_df_antilog['log_median_income_antilog'] = np.exp(
test_df_antilog['log_median_income'])

test_df_antilog.head()
# Test plot
# test_df_antilog.sort_values(by='log_median_income_antilog').plot(x='log_median_income_antilog', y='predicted',
#                      kind='scatter')

,index,log_median_income,measured,predicted,log_median_income_antilog
0,8,9.297527,0.511536,0.405743,10911.0
1,16,11.084402,0.265313,0.597056,65147.0
2,0,11.220405,0.619154,0.611618,74638.0
3,24,11.179199,0.596825,0.607206,71625.0
4,11,10.690535,0.677538,0.554887,43938.0


In [62]:
regression_diagnostic_plot = ((
    test_df
    .hvplot.scatter(x='measured', y='predicted')
    .opts(aspect='equal', xlim=(0, y_max), ylim=(0, y_max), width=600, height=600)
) * hv.Slope(slope=1, y_intercept=0).opts(color='black')).opts(
    width=800,
    height=600,
    title="Error in linear model prediction for median income versus greenspace fraction by census tract"
)

regression_diagnostic_plot

:Overlay
   .Scatter.I    :Scatter   [measured]   (predicted)
   .Annotation.I :Slope   [x,y]

This diagnostic plot of measured versus predicted values for the linear model shows that while the model is not great at making predictions, there does not seem to be any serious bias in the model predictions.

In [63]:
# Calculate and map spatial bias in the model predictions.

analysis_df['pred_fraction_greenspace'] = linear_reg.predict(X)
analysis_df['err_fraction_greenspace'] = analysis_df['pred_fraction_greenspace'] - analysis_df['fraction_greenspace']

tract_boundaries_gdf = tracts_w_census_gdf[['tract', 'geometry']]

analysis_gdf = pd.merge(tract_boundaries_gdf, analysis_df, on='tract')
analysis_gdf

(
    analysis_gdf.hvplot(geo=True, color='err_fraction_greenspace', cmap='RdBu')
    .redim.range(err_fraction_greenspace=(-.3, .3))
    .opts(frame_width=600, aspect='equal',
          title='Map of model error')
)


:Polygons   [Longitude,Latitude]   (err_fraction_greenspace)

This map shows whether the model is overpredicting (blue) or underpredicting (red) greenspace fraction in census tracts across the city. In the northern edge of the city, the model seems to be predicting higher greenspace than the true value, while in the western and southern tracts, the model seems to be predicting lower greenspace fraction than the true values.

### **Visualizing the model**


In [64]:
model_plot = (
    (test_df_antilog
     .sort_values(by='log_median_income_antilog')
     .hvplot.line(
    x='log_median_income_antilog',
    y='predicted',
    color='red')
    )
* scatter_plot).opts(xlabel='median annual household income', 
                     ylabel='fraction of greenspace',
                     title='Median annual income is not strongly correlated with higher fraction of greenspace \n'
                     'in Boulder, CO')


model_plot

:Overlay
   .Curve.I   :Curve   [log_median_income_antilog]   (predicted)
   .Scatter.I :Scatter   [median_income]   (fraction_greenspace)

Median income versus greenspace fraction by census tract in Boulder, CO with observed data (points) and model prediction (red line). Visual inspection of this plot shows that there is not a strong relationship between median income and greenspace fraction in Boulder, suggesting that greenspace is relatively evenly distributed across the city. 

However there are several caveats to this analytical approach that deserve mention. First, a rigorous statistical analysis was not carried out; future work could utilize formal hypothesis testing (using frequentist or Bayesian approaches) and explore other statistical models such as mixed models which would allow the analysis to control for variables that may be confounding (e.g., distance to natural forestland). Additionally, future work should investigate multiple socioeconomic variables as median income may not be the best metric to measure economic inequality. For example, household wealth may be a better proxy, as it represents multi-generational accumulation of income. Finally, the method used to calculate greenspace relies on a threshold for NDVI to classify a pixel as "greenspace." Future research should repeat this analysis with multiple NDVI thresholds to see how sensitive the results of the analysis are to the choice of threshold. A more rigorous analysis could also intersect the NDVI data with a layer of known landcover classes to calculate what NDVI values correspond to greenspace in Boulder and then derive a threshold for use in the greenspace analysis.

In [70]:
%%capture
%%bash
jupyter nbconvert boulder_urban_greenspace.ipynb --to html --no-input